# Load EQ series data to EQUITY table


## Table details:

*Since table creation is one time task and structure can be changed person to person. Adding create table scripts here.*

EQUITY - EQ series data are loaded here.


```SQL

-- Create table script

CREATE TABLE [dbo].[EQUITY](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[BHAVID] [int] NOT NULL,
	[SYMBOL] [nvarchar](255) NULL,
	[SERIES] [nvarchar](255) NULL,
	[OPEN] [float] NULL,
	[HIGH] [float] NULL,
	[LOW] [float] NULL,
	[CLOSE] [float] NULL,
	[LAST] [float] NULL,
	[PREVCLOSE] [float] NULL,
	[TOTTRDQTY] [float] NULL,
	[TOTTRDVAL] [float] NULL,
	[TIMESTAMP] [date] NULL,
	[TOTALTRADES] [float] NULL,
	[ISIN] [nvarchar](255) NULL,
	[DIFF] [float] NULL,
	[DIFFPREC] [float] NULL,
	[POSITIVE] [bit] NULL,
	[YEAR] [int] NULL,
	[MONTH] [int] NULL,
	[WEEKDAY] [int] NULL
) ON [PRIMARY]
```

In [4]:
# Import libraries

import pandas as pd
import sqlalchemy as sa
import pymssql

import numpy as np

In [5]:
# Server details

RDBMS_NAME = '<RDBMS NAME>'
DRIVER_NAME = '<DRIVER NAME>'
USER_NAME = '<USER NAME>'
PASSWORD = '<PASSWORD>'
SERVER_NAME = '<SERVER NAME>'
DATABASE_NAME = '<DATABASE NAME>'

In [6]:
# Table name for bhav_copy and test table

BHAV_COPY_TABLE_NAME = 'BHAV_COPY'
EQUITY_TABLE_NAME = 'EQUITY'

In [7]:
# Bhav_copy to Equity table cdc data sql query

cdc_sql = f''' 
SELECT * 
FROM
    {BHAV_COPY_TABLE_NAME} 
WHERE 
    "TIMESTAMP"  > (SELECT ISNULL(MAX("timestamp"),'2000-01-01') FROM {EQUITY_TABLE_NAME} ) 
    AND "SERIES" = 'EQ'   '''

In [8]:
# Validation query for cdc

bhav_copy_count_sql = f''' SELECT COUNT(*) AS "COUNT" FROM {BHAV_COPY_TABLE_NAME} WHERE "SERIES" = 'EQ' '''
equity_count_sql = f''' SELECT COUNT(*) AS "COUNT" FROM {EQUITY_TABLE_NAME}  '''

In [9]:
# Database connection

engine = sa.create_engine(f'{RDBMS_NAME}+{DRIVER_NAME}://{USER_NAME}:{PASSWORD}@{SERVER_NAME}/{DATABASE_NAME}')

In [10]:
with engine.connect() as con:
    cdc_df = pd.read_sql(cdc_sql, con, parse_dates=['TIMESTAMP'])
    cdc_df['DIFF'] = (cdc_df['PREVCLOSE'] - cdc_df['CLOSE']).round(3)
    cdc_df['DIFFPREC'] = ((cdc_df['PREVCLOSE'] - cdc_df['CLOSE'])/cdc_df['PREVCLOSE']).round(6)
    cdc_df['POSITIVE'] = cdc_df['DIFF'] > 0
    cdc_df['YEAR'] = cdc_df['TIMESTAMP'].dt.year
    cdc_df['MONTH'] = cdc_df['TIMESTAMP'].dt.month
    cdc_df['WEEKDAY'] = cdc_df['TIMESTAMP'].dt.weekday
    
    cdc_df = cdc_df.replace([np.inf, -np.inf], np.nan)
    
    cdc_df.drop(columns=['FNAME'], inplace=True)
    
    cdc_df.rename(columns = {'ID': 'BHAVID'}, inplace=True)
    
    print('CDC data fetched')
    
    cdc_df.to_sql(EQUITY_TABLE_NAME, con, if_exists='append', chunksize=100000, index=False) # Loading chunk by chunk
    
    print("CDC data loaded.")

CDC data fetched
CDC data loaded.


In [13]:
with engine.connect() as con:
    master_count = pd.read_sql(bhav_copy_count_sql, con)
    equity_count = pd.read_sql(equity_count_sql, con)
    
    result = (master_count == equity_count)
    
    if result.all(axis=None):
        print("CDC validation pass")
    else:
        print("CDC validation failed")
        print(f'Master table - {master_count}')
        print(f'Equity table - {equity_count}')

print('Program ends')

CDC validation pass
Program ends
